In [22]:
import pandas 
import os

In [23]:
logements_neufs = pandas.read_csv('../Data/dpe-v2-logements-neufs.csv')
logements_existants = pandas.read_csv('../Data/dpe-v2-logements-existants.csv')
# logements_neufs = pandas.read_csv('../Data/neufs_69.csv')
# logements_existants = pandas.read_csv('../Data/existant_69.csv')
adresses = pandas.read_csv('../Data/adresses-69.csv', sep=";")

C:\Users\ediad\AppData\Local\Temp\ipykernel_18852\4093467828.py:2: DtypeWarning: Columns (12,48,190,191,192,193) have mixed types. Specify dtype option on import or set low_memory=False.
  logements_existants = pandas.read_csv('../Data/dpe-v2-logements-existants.csv')
C:\Users\ediad\AppData\Local\Temp\ipykernel_18852\4093467828.py:5: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  adresses = pandas.read_csv('../Data/adresses-69.csv', sep=";")


In [24]:
# Ajouter la colonne 'Logement' avec la valeur 'neuf' dans le dataset logements_neufs
logements_neufs['Logement'] = 'neuf'

# Ajouter la colonne 'Logement' avec la valeur 'ancien' dans le dataset logements_existants
logements_existants['Logement'] = 'ancien'


In [25]:
import datetime

# Obtenir l'année en cours
annee_courante = datetime.datetime.now().year

# Ajouter la colonne 'Année_construction' dans le dataset logements_neufs avec la valeur de l'année courante
logements_neufs['Année_construction'] = annee_courante


In [26]:
# Étape 1 : Lister les colonnes des deux DataFrames
colonnes_communes = logements_neufs.columns.intersection(logements_existants.columns)

# Étape 3 : Sélectionner uniquement les colonnes communes
logements_neufs_communs = logements_neufs[colonnes_communes]
logements_existants_communs = logements_existants[colonnes_communes]

# Étape 4 : Concaténer les DataFrames sur les colonnes communes
df = pandas.concat([logements_neufs_communs, logements_existants_communs])

# Afficher les dimensions du DataFrame fusionné pour vérifier
df


,N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),...,Electricité_PV_autoconsommée,Système_production_électricité_origine_renouvelable,Présence_production_PV_(0/1),Surface_totale_capteurs_photovoltaïque,Nombre_module,Type_énergie_principale_chauffage,Type_énergie_principale_ECS,Conso_5_usages_é_finale_énergie_n°1,Logement,Année_construction
0,2469N0892797M,2024-03-12,2024-03-11,2024-03-10,DPE NEUF logement : RT2012,NaN,2034-03-10,2.3,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,2053.5,neuf,2024.0
1,2469N0892783Y,2024-03-12,2024-03-11,2024-03-10,DPE NEUF logement : RT2012,NaN,2034-03-10,2.3,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,2011.2,neuf,2024.0
2,2469N0892780V,2024-03-12,2024-03-11,2024-03-10,DPE NEUF logement : RT2012,NaN,2034-03-10,2.3,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,2408.2,neuf,2024.0
3,2469N0892802R,2024-03-12,2024-03-11,2024-03-10,DPE NEUF logement : RT2012,NaN,2034-03-10,2.3,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,2092.5,neuf,2024.0
4,2469N2757372X,2024-07-30,2024-07-29,2024-07-07,DPE NEUF logement : RT2012,NaN,2034-07-28,2.4,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,3042.7,neuf,2024.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16064,2369E2954339E,2023-09-06,2023-05-23,2023-05-23,DPE 3CL 2021 méthode logement,2369E2085485S,2033-05-22,2.3,NaN,0.0,...,NaN,Solaire Photovoltaïque,0.0,NaN,NaN,Gaz naturel,Gaz naturel,8152.5,ancien,1948.0
16065,2369E2586828D,2023-07-31,2023-07-30,2023-07-30,DPE 3CL 2021 méthode logement,NaN,2033-07-29,2.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Gaz naturel,Gaz naturel,6850.2,ancien,NaN
16066,2369E3098145E,2023-09-18,2023-09-17,2023-09-11,DPE 3CL 2021 méthode logement,2369E3035921Y,2033-09-16,2.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Électricité,Électricité,3917.2,ancien,1980.0
16067,2369E2586714T,2023-07-31,2023-07-30,2023-07-30,DPE 3CL 2021 méthode logement,NaN,2033-07-29,2.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Gaz naturel,Gaz naturel,6277.9,ancien,NaN


In [27]:
# Jointure sur le champ Identifiant__BAN pour ajouter les coordonnées GPS (latitude et longitude)
df_merged = pandas.merge(df, adresses[['id', 'lat', 'lon']], 
                     left_on='Identifiant__BAN', right_on='id', how='left')

# Vérification des premières lignes après la jointure
diff_id_ban = df['Identifiant__BAN'].isin(adresses['id']) == False

# Combine results to count different values
different_values_count = diff_id_ban.sum()
print(different_values_count)
print(df_merged[['id', 'lat', 'lon']].isna().sum())

1216
id     1216
lat    1216
lon    1216
dtype: int64


In [28]:
df_merged.shape

(18920, 199)

In [29]:
df_merged =  df_merged[['lat',  'lon','Logement','Coût_chauffage','Surface_habitable_logement']]

In [30]:
df_merged.shape

(18920, 5)

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create the base map centered around Lyon (or any other location)
lyon_lat, lyon_lon = 45.75, 4.85
carte_logements = folium.Map(location=[lyon_lat, lyon_lon], zoom_start=12)

# Initialize the MarkerCluster object
marker_cluster = MarkerCluster().add_to(carte_logements)

# Add the markers with clustering
for idx, row in df_merged.iterrows():
    if pandas.notna(row['lat']) and pandas.notna(row['lon']):
        # Create a popup text for each marker
        popup_text = f"Logement: {row['Logement']}<br>Coût chauffage: {row['Coût_chauffage']}€<br>Surface: {row['Surface_habitable_logement']} m²"
        
        # Create a marker for each property and add it to the marker cluster
        folium.Marker(
            location=[row['lat'], row['lon']],
            popup=popup_text
        ).add_to(marker_cluster)

# Display the map
carte_logements


In [ ]:
carte_logements

In [ ]:
import pickle


# Save the map to a pickle file
with open('carte_logements.pkl', 'wb') as f:
    pickle.dump(carte_logements, f)

# To load the map back later
with open('carte_logements.pkl', 'rb') as f:
    loaded_map = pickle.load(f)

# Display the loaded map (in a Jupyter Notebook, for example)
loaded_map